# Import the dataset and explore the data

In [176]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# data partition
from sklearn.model_selection import train_test_split

#filter methods
# spearman 
# chi-square
import scipy.stats as stats
from scipy.stats import chi2_contingency

#wrapper methods
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import RFE

# embedded methods
from sklearn.linear_model import LassoCV

from sklearn.metrics import confusion_matrix

In [177]:
#First Hearing Date: blank means no hearing yet, can we input? I think not
#Agreement Reached: Can't input
#WCB Decision: Can't input

In [178]:
train_data = pd.read_csv('train_data.csv', sep=';')
test_data = pd.read_csv('test_data.csv', sep=',')
#what does this warning mean?

C:\Users\lopes\AppData\Local\Temp\ipykernel_16892\1873223536.py:1: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('train_data.csv', sep=';')


In [179]:
train_data.head()
#Claim Identifier: should be the first column
#Claim Injury Type: should be the last column
#line 3 just have the claim identifier and assemble date, we should delete it

,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,Claim Identifier,Claim Injury Type,County of Injury,COVID-19 Indicator,District Name,First Hearing Date,Gender,IME-4 Count,Industry Code,Industry Code Description,Medical Fee Region,OIICS Nature of Injury Description,WCIO Cause of Injury Code,WCIO Cause of Injury Description,WCIO Nature of Injury Code,WCIO Nature of Injury Description,WCIO Part Of Body Code,WCIO Part Of Body Description,Zip Code,Agreement Reached,WCB Decision,Number of Dependents
0,30/12/2019,31.0,N,01/01/2020,N,0.00,1988.0,31/12/2019,NaN,NEW HAMPSHIRE INSURANCE CO,1A. PRIVATE,5393875,2. NON-COMP,ST. LAWRENCE,N,SYRACUSE,NaN,M,NaN,44.0,RETAIL TRADE,I,NaN,27.0,FROM LIQUID OR GREASE SPILLS,10.0,CONTUSION,62.0,BUTTOCKS,13662,0.0,Not Work Related,1.0
1,30/08/2019,46.0,N,01/01/2020,Y,1745.93,1973.0,01/01/2020,14/01/2020,ZURICH AMERICAN INSURANCE CO,1A. PRIVATE,5393091,4. TEMPORARY,WYOMING,N,ROCHESTER,21/02/2020,F,4.0,23.0,CONSTRUCTION,I,NaN,97.0,REPETITIVE MOTION,49.0,SPRAIN OR TEAR,38.0,SHOULDER(S),14569,1.0,Not Work Related,4.0
2,06/12/2019,40.0,N,01/01/2020,N,1434.80,1979.0,01/01/2020,NaN,INDEMNITY INSURANCE CO OF,1A. PRIVATE,5393889,4. TEMPORARY,ORANGE,N,ALBANY,NaN,M,NaN,56.0,ADMINISTRATIVE AND SUPPORT AND WASTE MANAGEMEN...,II,NaN,79.0,OBJECT BEING LIFTED OR HANDLED,7.0,CONCUSSION,10.0,MULTIPLE HEAD INJURY,12589,0.0,Not Work Related,6.0
3,NaN,NaN,NaN,01/01/2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,957648180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30/12/2019,61.0,N,01/01/2020,N,NaN,1958.0,31/12/2019,NaN,STATE INSURANCE FUND,2A. SIF,5393887,2. NON-COMP,DUTCHESS,N,ALBANY,NaN,M,NaN,62.0,HEALTH CARE AND SOCIAL ASSISTANCE,II,NaN,16.0,"HAND TOOL, UTENSIL",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [180]:
train_data.info()
#OIICS Nature of Injury Description: no data
#IME-4 Count: Few info, just for who has an attorney? maybe we should delete for modeling purposes
#C-3 Date: very few values 1/5 of the data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 593471 entries, 0 to 593470
Data columns (total 33 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   Accident Date                       570337 non-null  object 
 1   Age at Injury                       574026 non-null  float64
 2   Alternative Dispute Resolution      574026 non-null  object 
 3   Assembly Date                       593471 non-null  object 
 4   Attorney/Representative             574026 non-null  object 
 5   Average Weekly Wage                 545375 non-null  float64
 6   Birth Year                          544948 non-null  float64
 7   C-2 Date                            559466 non-null  object 
 8   C-3 Date                            187245 non-null  object 
 9   Carrier Name                        574026 non-null  object 
 10  Carrier Type                        574026 non-null  object 
 11  Claim Identifier          

In [181]:
train_data.describe()

,Age at Injury,Average Weekly Wage,Birth Year,Claim Identifier,IME-4 Count,Industry Code,OIICS Nature of Injury Description,WCIO Cause of Injury Code,WCIO Nature of Injury Code,WCIO Part Of Body Code,Agreement Reached,Number of Dependents
count,574026.000000,5.453750e+05,544948.000000,5.934710e+05,132803.000000,564068.000000,0.0,558386.000000,548447.000000,547042.000000,564103.000000,564103.000000
mean,42.114270,4.910883e+02,1886.767604,2.366760e+07,3.207337,58.645305,NaN,54.381143,41.014960,39.804907,0.047303,3.006426
std,14.256432,6.092918e+03,414.644423,1.079271e+08,2.832303,19.644175,NaN,25.874281,22.387101,22.539086,0.212287,2.001031
min,0.000000,0.000000e+00,0.000000,5.393066e+06,1.000000,11.000000,NaN,1.000000,1.000000,-9.000000,0.000000,0.000000
25%,31.000000,0.000000e+00,1965.000000,5.593414e+06,1.000000,45.000000,NaN,31.000000,16.000000,33.000000,0.000000,1.000000
50%,42.000000,0.000000e+00,1977.000000,5.791212e+06,2.000000,61.000000,NaN,56.000000,49.000000,38.000000,0.000000,3.000000
75%,54.000000,8.410000e+02,1989.000000,5.991000e+06,4.000000,71.000000,NaN,75.000000,52.000000,53.000000,0.000000,5.000000
max,117.000000,2.828079e+06,2018.000000,9.998917e+08,73.000000,92.000000,NaN,99.000000,91.000000,99.000000,1.000000,6.000000


In [182]:
columns_name = train_data.columns
columns_name

Index(['Accident Date', 'Age at Injury', 'Alternative Dispute Resolution',
       'Assembly Date', 'Attorney/Representative', 'Average Weekly Wage',
       'Birth Year', 'C-2 Date', 'C-3 Date', 'Carrier Name', 'Carrier Type',
       'Claim Identifier', 'Claim Injury Type', 'County of Injury',
       'COVID-19 Indicator', 'District Name', 'First Hearing Date', 'Gender',
       'IME-4 Count', 'Industry Code', 'Industry Code Description',
       'Medical Fee Region', 'OIICS Nature of Injury Description',
       'WCIO Cause of Injury Code', 'WCIO Cause of Injury Description',
       'WCIO Nature of Injury Code', 'WCIO Nature of Injury Description',
       'WCIO Part Of Body Code', 'WCIO Part Of Body Description', 'Zip Code',
       'Agreement Reached', 'WCB Decision', 'Number of Dependents'],
      dtype='object')

In [183]:
for i in columns_name:
    print(i)
    print(train_data[i].unique())
    print(train_data[i].value_counts())
    print('\n')
#Age at Injury: if it's a workplace it means ages like 5 or 9 don't make sense, we should delete them
#Alternative Dispute Resolution: what the values mean? ['N' nan 'Y' 'U']
#Average Weekly Wage: make sense they don't receive money?
#Birth Year: with a lot of 0.0
#Claim Identifier: 292668076 is repeated
#Gender: what is 'U' or 'X'?
#WCIO Part Of Body Code: -9 is an error?
#Zip Code: 'G9B0Y' is an error?
#WCB Decision: just 'Not Work Related' and 'nan'? maybe we can delete this column

#maybe 0.0 it's a nan, some values we don't have, but exists
#Carrier Type: we have a lot of UNKNOW, so maybe we can conclude 0.0 or nothing mean missing values

Accident Date
['30/12/2019' '30/08/2019' '06/12/2019' ... '24/05/2000' '01/05/2007'
 '17/05/1980']
Accident Date
01/03/2020    1245
18/12/2020    1001
07/02/2022     977
05/01/2022     883
18/02/2021     851
              ... 
17/05/2017       1
28/05/2015       1
18/08/2002       1
01/09/2017       1
17/05/1980       1
Name: count, Length: 5539, dtype: int64


Age at Injury
[ 31.  46.  40.  nan  61.  67.  48.  33.  55.  20.  21.  51.  62.  35.
  54.  32.  34.  38.  30.  36.  39.  44.  56.  29.  60.  49.  50.  18.
  58.  24.  45.  53.  52.  47.  25.  23.  22.  64.  70.  41.  69.  57.
  27.  42.  77.  72.  19.  28.  63.  65.  59.  43.  26.  74.  66.  76.
  68.  37.   0.  16.  71.  82.  75.  17.  78.  73.  89.  85.  84.  81.
  87.  80.  11.  88.  79.  15.  83.  86.  90.   1.  10. 102.  94. 117.
  14. 113.  95.  92. 104.   5.   8. 115. 110.  91.  96.  99.  12.  13.
  93. 101.   7. 111.   9.  97. 112. 109. 100.  98. 114.]
Age at Injury
31.0     14041
30.0     14022
32.0     13994
29.0     

# Clean and pre-process the dataset

In [184]:
train_data.shape[0] #number of rows, the same as Claim Identifier

593471

In [185]:
train_data.set_index('Claim Identifier', inplace=True) #set Claim Identifier as index

In [186]:
train_data.head()

,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,Claim Injury Type,County of Injury,COVID-19 Indicator,District Name,First Hearing Date,Gender,IME-4 Count,Industry Code,Industry Code Description,Medical Fee Region,OIICS Nature of Injury Description,WCIO Cause of Injury Code,WCIO Cause of Injury Description,WCIO Nature of Injury Code,WCIO Nature of Injury Description,WCIO Part Of Body Code,WCIO Part Of Body Description,Zip Code,Agreement Reached,WCB Decision,Number of Dependents
Claim Identifier,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5393875,30/12/2019,31.0,N,01/01/2020,N,0.00,1988.0,31/12/2019,NaN,NEW HAMPSHIRE INSURANCE CO,1A. PRIVATE,2. NON-COMP,ST. LAWRENCE,N,SYRACUSE,NaN,M,NaN,44.0,RETAIL TRADE,I,NaN,27.0,FROM LIQUID OR GREASE SPILLS,10.0,CONTUSION,62.0,BUTTOCKS,13662,0.0,Not Work Related,1.0
5393091,30/08/2019,46.0,N,01/01/2020,Y,1745.93,1973.0,01/01/2020,14/01/2020,ZURICH AMERICAN INSURANCE CO,1A. PRIVATE,4. TEMPORARY,WYOMING,N,ROCHESTER,21/02/2020,F,4.0,23.0,CONSTRUCTION,I,NaN,97.0,REPETITIVE MOTION,49.0,SPRAIN OR TEAR,38.0,SHOULDER(S),14569,1.0,Not Work Related,4.0
5393889,06/12/2019,40.0,N,01/01/2020,N,1434.80,1979.0,01/01/2020,NaN,INDEMNITY INSURANCE CO OF,1A. PRIVATE,4. TEMPORARY,ORANGE,N,ALBANY,NaN,M,NaN,56.0,ADMINISTRATIVE AND SUPPORT AND WASTE MANAGEMEN...,II,NaN,79.0,OBJECT BEING LIFTED OR HANDLED,7.0,CONCUSSION,10.0,MULTIPLE HEAD INJURY,12589,0.0,Not Work Related,6.0
957648180,NaN,NaN,NaN,01/01/2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5393887,30/12/2019,61.0,N,01/01/2020,N,NaN,1958.0,31/12/2019,NaN,STATE INSURANCE FUND,2A. SIF,2. NON-COMP,DUTCHESS,N,ALBANY,NaN,M,NaN,62.0,HEALTH CARE AND SOCIAL ASSISTANCE,II,NaN,16.0,"HAND TOOL, UTENSIL",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [187]:
pd.set_option('display.max_columns', None)
train_data[train_data.index == 292668076] #to delete, no info

,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,Claim Injury Type,County of Injury,COVID-19 Indicator,District Name,First Hearing Date,Gender,IME-4 Count,Industry Code,Industry Code Description,Medical Fee Region,OIICS Nature of Injury Description,WCIO Cause of Injury Code,WCIO Cause of Injury Description,WCIO Nature of Injury Code,WCIO Nature of Injury Description,WCIO Part Of Body Code,WCIO Part Of Body Description,Zip Code,Agreement Reached,WCB Decision,Number of Dependents
Claim Identifier,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
292668076,NaN,NaN,NaN,21/05/2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
292668076,NaN,NaN,NaN,05/09/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [188]:
train_data.drop(292668076, inplace=True) #delete the rows
train_data.drop('OIICS Nature of Injury Description', axis=1, inplace=True) #delete the column, just nan values
train_data = train_data[train_data['Age at Injury'] >= 16] #can't work if they are younger than 16
train_data.head()

,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,Claim Injury Type,County of Injury,COVID-19 Indicator,District Name,First Hearing Date,Gender,IME-4 Count,Industry Code,Industry Code Description,Medical Fee Region,WCIO Cause of Injury Code,WCIO Cause of Injury Description,WCIO Nature of Injury Code,WCIO Nature of Injury Description,WCIO Part Of Body Code,WCIO Part Of Body Description,Zip Code,Agreement Reached,WCB Decision,Number of Dependents
Claim Identifier,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5393875,30/12/2019,31.0,N,01/01/2020,N,0.00,1988.0,31/12/2019,NaN,NEW HAMPSHIRE INSURANCE CO,1A. PRIVATE,2. NON-COMP,ST. LAWRENCE,N,SYRACUSE,NaN,M,NaN,44.0,RETAIL TRADE,I,27.0,FROM LIQUID OR GREASE SPILLS,10.0,CONTUSION,62.0,BUTTOCKS,13662,0.0,Not Work Related,1.0
5393091,30/08/2019,46.0,N,01/01/2020,Y,1745.93,1973.0,01/01/2020,14/01/2020,ZURICH AMERICAN INSURANCE CO,1A. PRIVATE,4. TEMPORARY,WYOMING,N,ROCHESTER,21/02/2020,F,4.0,23.0,CONSTRUCTION,I,97.0,REPETITIVE MOTION,49.0,SPRAIN OR TEAR,38.0,SHOULDER(S),14569,1.0,Not Work Related,4.0
5393889,06/12/2019,40.0,N,01/01/2020,N,1434.80,1979.0,01/01/2020,NaN,INDEMNITY INSURANCE CO OF,1A. PRIVATE,4. TEMPORARY,ORANGE,N,ALBANY,NaN,M,NaN,56.0,ADMINISTRATIVE AND SUPPORT AND WASTE MANAGEMEN...,II,79.0,OBJECT BEING LIFTED OR HANDLED,7.0,CONCUSSION,10.0,MULTIPLE HEAD INJURY,12589,0.0,Not Work Related,6.0
5393887,30/12/2019,61.0,N,01/01/2020,N,NaN,1958.0,31/12/2019,NaN,STATE INSURANCE FUND,2A. SIF,2. NON-COMP,DUTCHESS,N,ALBANY,NaN,M,NaN,62.0,HEALTH CARE AND SOCIAL ASSISTANCE,II,16.0,"HAND TOOL, UTENSIL",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5393863,26/12/2019,67.0,N,01/01/2020,N,0.00,1952.0,31/12/2019,NaN,INDEMNITY INS. OF N AMERICA,1A. PRIVATE,3. MED ONLY,SUFFOLK,N,HAUPPAUGE,NaN,M,NaN,44.0,RETAIL TRADE,IV,31.0,"FALL, SLIP OR TRIP, NOC",10.0,CONTUSION,38.0,SHOULDER(S),11772,0.0,Not Work Related,5.0


In [189]:
pd.reset_option('display.max_columns')

In [190]:
X = train_data.drop('Claim Injury Type', axis = 1)
y = train_data['Claim Injury Type']

In [221]:
X_train, X_validation,y_train, y_validation = train_test_split(X,y,
                                                               train_size = 0.7,
                                                               random_state=0,
                                                               shuffle = True, 
                                                               stratify = y)

In [192]:
""" #filling with mean
for column in X_train.columns:
    if pd.api.types.is_numeric_dtype(X_train[column]):
        
        #store mean of training data in a variable - in a real application, you may need to store these values for future usages on e.g. test data 
        mean_to_fill = X_train[column].mean()
        
        #fill on X_train
        X_train[column].fillna(mean_to_fill, inplace=True) #X_train[column].fillna('Unknown')
        #Fill on X_val
        X_val[column].fillna(mean_to_fill, inplace=True) """

" #filling with mean\nfor column in X_train.columns:\n    if pd.api.types.is_numeric_dtype(X_train[column]):\n        \n        #store mean of training data in a variable - in a real application, you may need to store these values for future usages on e.g. test data \n        mean_to_fill = X_train[column].mean()\n        \n        #fill on X_train\n        X_train[column].fillna(mean_to_fill, inplace=True) #X_train[column].fillna('Unknown')\n        #Fill on X_val\n        X_val[column].fillna(mean_to_fill, inplace=True) "

In [193]:
""" X_train_num = X_train.select_dtypes(include=np.number).set_index(X_train.index)
X_train_num.drop('Embarked_Flag', axis = 1, inplace = True)

X_train_cat = X_train.select_dtypes(exclude=np.number).set_index(X_train.index)
X_train_cat['Embarked_Flag'] = X_train['Embarked_Flag']

# Repeat for Validation
#CODE HERE
X_val_num = X_val.select_dtypes(include=np.number).set_index(X_val.index)
X_val_num.drop('Embarked_Flag', axis = 1, inplace = True)

X_val_cat = X_val.select_dtypes(exclude=np.number).set_index(X_val.index)
X_val_cat['Embarked_Flag'] = X_val['Embarked_Flag'] """

" X_train_num = X_train.select_dtypes(include=np.number).set_index(X_train.index)\nX_train_num.drop('Embarked_Flag', axis = 1, inplace = True)\n\nX_train_cat = X_train.select_dtypes(exclude=np.number).set_index(X_train.index)\nX_train_cat['Embarked_Flag'] = X_train['Embarked_Flag']\n\n# Repeat for Validation\n#CODE HERE\nX_val_num = X_val.select_dtypes(include=np.number).set_index(X_val.index)\nX_val_num.drop('Embarked_Flag', axis = 1, inplace = True)\n\nX_val_cat = X_val.select_dtypes(exclude=np.number).set_index(X_val.index)\nX_val_cat['Embarked_Flag'] = X_val['Embarked_Flag'] "

In [194]:
""" #imported here for demonstrative purposes but it is a good practice to import everything at the start.
from sklearn.preprocessing import MinMaxScaler

#call function
scaler = MinMaxScaler()

#fit to training data
scaler.fit(X_train_num)

#transform the data
X_train_num_scaled = scaler.transform(X_train_num) # this will return an array

#show results
X_train_num_scaled """

' #imported here for demonstrative purposes but it is a good practice to import everything at the start.\nfrom sklearn.preprocessing import MinMaxScaler\n\n#call function\nscaler = MinMaxScaler()\n\n#fit to training data\nscaler.fit(X_train_num)\n\n#transform the data\nX_train_num_scaled = scaler.transform(X_train_num) # this will return an array\n\n#show results\nX_train_num_scaled '

In [195]:
""" # Convert the array to a pandas dataframe
X_train_num_scaled = pd.DataFrame(X_train_num_scaled, columns = X_train_num.columns).set_index(X_train.index)
X_train_num_scaled """

' # Convert the array to a pandas dataframe\nX_train_num_scaled = pd.DataFrame(X_train_num_scaled, columns = X_train_num.columns).set_index(X_train.index)\nX_train_num_scaled '

In [196]:
""" X_val_num_scaled = scaler.transform(X_val_num)
X_val_num_scaled = pd.DataFrame(X_val_num_scaled, columns = X_val_num.columns).set_index(X_val.index)
X_val_num_scaled.head(3) """

' X_val_num_scaled = scaler.transform(X_val_num)\nX_val_num_scaled = pd.DataFrame(X_val_num_scaled, columns = X_val_num.columns).set_index(X_val.index)\nX_val_num_scaled.head(3) '

# Feature selection

In [197]:
""" X_train_num_scaled.var() """

' X_train_num_scaled.var() '

In [198]:
""" def cor_heatmap(cor):
    plt.figure(figsize=(12,10))
    sns.heatmap(data = cor, annot = True, cmap = plt.cm.Reds, fmt='.1')
    plt.show() """

" def cor_heatmap(cor):\n    plt.figure(figsize=(12,10))\n    sns.heatmap(data = cor, annot = True, cmap = plt.cm.Reds, fmt='.1')\n    plt.show() "

In [199]:
""" cor_spearman = X_train_num_scaled.corr(method ='spearman')
cor_spearman """

" cor_spearman = X_train_num_scaled.corr(method ='spearman')\ncor_spearman "

In [200]:
""" cor_heatmap(cor_spearman) """

' cor_heatmap(cor_spearman) '

In [201]:
""" def TestIndependence(X,y,var,alpha=0.05):        
    dfObserved = pd.crosstab(y,X) 
    chi2, p, dof, expected = stats.chi2_contingency(dfObserved.values)
    dfExpected = pd.DataFrame(expected, columns=dfObserved.columns, index = dfObserved.index)
    if p<alpha:
        result="{0} is IMPORTANT for Prediction".format(var)
    else:
        result="{0} is NOT an important predictor. (Discard {0} from model)".format(var)
    print(result) """

' def TestIndependence(X,y,var,alpha=0.05):        \n    dfObserved = pd.crosstab(y,X) \n    chi2, p, dof, expected = stats.chi2_contingency(dfObserved.values)\n    dfExpected = pd.DataFrame(expected, columns=dfObserved.columns, index = dfObserved.index)\n    if p<alpha:\n        result="{0} is IMPORTANT for Prediction".format(var)\n    else:\n        result="{0} is NOT an important predictor. (Discard {0} from model)".format(var)\n    print(result) '

In [202]:
""" for var in X_train_cat:
    TestIndependence(X_train_cat[var],y_train, var) """

' for var in X_train_cat:\n    TestIndependence(X_train_cat[var],y_train, var) '

In [203]:
""" model = LogisticRegression()
rfe = RFE(estimator = model, n_features_to_select = 3)
X_rfe = rfe.fit_transform(X = X_train_num_scaled, y = y_train)
X_train_num_scaled.columns """

' model = LogisticRegression()\nrfe = RFE(estimator = model, n_features_to_select = 3)\nX_rfe = rfe.fit_transform(X = X_train_num_scaled, y = y_train)\nX_train_num_scaled.columns '

In [204]:
""" rfe.support_ """

' rfe.support_ '

In [205]:
""" rfe.ranking_ """

' rfe.ranking_ '

In [206]:
""" selected_features = pd.Series(rfe.support_, index = X_train_num_scaled.columns)
selected_features """

' selected_features = pd.Series(rfe.support_, index = X_train_num_scaled.columns)\nselected_features '

In [207]:
""" #no of features
nof_list=np.arange(1,8)            
high_score=0
#Variable to store the optimum features
nof=0           
train_score_list =[]
val_score_list = []

for n in range(len(nof_list)):
    model = LogisticRegression()
    
    rfe = RFE(estimator = model,n_features_to_select = nof_list[n])
    X_train_rfe = rfe.fit_transform(X_train_num_scaled,y_train)
    X_val_rfe = rfe.transform(X_val_num_scaled)
    model.fit(X_train_rfe,y_train)
    
    #storing results on training data
    train_score = model.score(X_train_rfe,y_train)
    train_score_list.append(train_score)
    
    #storing results on training data
    val_score = model.score(X_val_rfe,y_val)
    val_score_list.append(val_score)
    
    #check best score
    if(val_score >= high_score):
        high_score = val_score
        nof = nof_list[n]
print("Optimum number of features: %d" %nof)
print("Score with %d features: %f" % (nof, high_score)) """

' #no of features\nnof_list=np.arange(1,8)            \nhigh_score=0\n#Variable to store the optimum features\nnof=0           \ntrain_score_list =[]\nval_score_list = []\n\nfor n in range(len(nof_list)):\n    model = LogisticRegression()\n    \n    rfe = RFE(estimator = model,n_features_to_select = nof_list[n])\n    X_train_rfe = rfe.fit_transform(X_train_num_scaled,y_train)\n    X_val_rfe = rfe.transform(X_val_num_scaled)\n    model.fit(X_train_rfe,y_train)\n    \n    #storing results on training data\n    train_score = model.score(X_train_rfe,y_train)\n    train_score_list.append(train_score)\n    \n    #storing results on training data\n    val_score = model.score(X_val_rfe,y_val)\n    val_score_list.append(val_score)\n    \n    #check best score\n    if(val_score >= high_score):\n        high_score = val_score\n        nof = nof_list[n]\nprint("Optimum number of features: %d" %nof)\nprint("Score with %d features: %f" % (nof, high_score)) '

In [208]:
""" plt.plot(list(range(1,8)), train_score_list, label="Score on Training Set", color='yellowgreen')
plt.plot(list(range(1,8)), val_score_list, label="Score on Test Set", color='dimgray')
plt.xlabel("Maximum Depth")
plt.ylabel("Score")
plt.legend()
plt.show() """

' plt.plot(list(range(1,8)), train_score_list, label="Score on Training Set", color=\'yellowgreen\')\nplt.plot(list(range(1,8)), val_score_list, label="Score on Test Set", color=\'dimgray\')\nplt.xlabel("Maximum Depth")\nplt.ylabel("Score")\nplt.legend()\nplt.show() '

In [209]:
""" #no of features
nof_list=np.arange(1,8)            
high_score=0
#Variable to store the optimum features
nof=0           
train_score_list =[]
val_score_list = []

for n in range(len(nof_list)):
    #call support vector machines classifier
    model = SVC(kernel = 'linear')
    
    #create RFE instance
    rfe = RFE(estimator = model,n_features_to_select = nof_list[n])
    
    #fit and transform for training data, transform for val data
    X_train_rfe = rfe.fit_transform(X_train_num_scaled,y_train)
    X_val_rfe = rfe.transform(X_val_num_scaled)
    
    model.fit(X_train_rfe,y_train)
    
    #storing results on training data
    train_score = model.score(X_train_rfe,y_train)
    train_score_list.append(train_score)
    
    #storing results on training data
    val_score = model.score(X_val_rfe,y_val)
    val_score_list.append(val_score)
    
    #check best score
    if(val_score > high_score):
        high_score = val_score
        nof = nof_list[n]
        
        #adding mention of variables to keep
        features_to_select = pd.Series(rfe.support_, index = X_train_num_scaled.columns)
        
print("Optimum number of features: %d" %nof)
print("Score with %d features: %f" % (nof, high_score))
print(f"Features to select: \n{features_to_select}") """

' #no of features\nnof_list=np.arange(1,8)            \nhigh_score=0\n#Variable to store the optimum features\nnof=0           \ntrain_score_list =[]\nval_score_list = []\n\nfor n in range(len(nof_list)):\n    #call support vector machines classifier\n    model = SVC(kernel = \'linear\')\n    \n    #create RFE instance\n    rfe = RFE(estimator = model,n_features_to_select = nof_list[n])\n    \n    #fit and transform for training data, transform for val data\n    X_train_rfe = rfe.fit_transform(X_train_num_scaled,y_train)\n    X_val_rfe = rfe.transform(X_val_num_scaled)\n    \n    model.fit(X_train_rfe,y_train)\n    \n    #storing results on training data\n    train_score = model.score(X_train_rfe,y_train)\n    train_score_list.append(train_score)\n    \n    #storing results on training data\n    val_score = model.score(X_val_rfe,y_val)\n    val_score_list.append(val_score)\n    \n    #check best score\n    if(val_score > high_score):\n        high_score = val_score\n        nof = nof_l

In [210]:
""" plt.plot(list(range(1,8)), train_score_list, label="Score on Training Set", color='yellowgreen')
plt.plot(list(range(1,8)), val_score_list, label="Score on Test Set", color='dimgray')
plt.xlabel("Maximum Depth")
plt.ylabel("Score")
plt.legend()
plt.show() """

' plt.plot(list(range(1,8)), train_score_list, label="Score on Training Set", color=\'yellowgreen\')\nplt.plot(list(range(1,8)), val_score_list, label="Score on Test Set", color=\'dimgray\')\nplt.xlabel("Maximum Depth")\nplt.ylabel("Score")\nplt.legend()\nplt.show() '

In [211]:
""" def plot_importance(coef,name):
    imp_coef = coef.sort_values()
    plt.figure(figsize=(8,10))
    imp_coef.plot(kind = "barh")
    plt.title("Feature importance using " + name + " Model")
    plt.show() """

' def plot_importance(coef,name):\n    imp_coef = coef.sort_values()\n    plt.figure(figsize=(8,10))\n    imp_coef.plot(kind = "barh")\n    plt.title("Feature importance using " + name + " Model")\n    plt.show() '

In [212]:
""" reg = LassoCV()
reg.fit(X_train_num_scaled, y_train)
coef = pd.Series(reg.coef_, index = X_train_num_scaled.columns)
coef """

' reg = LassoCV()\nreg.fit(X_train_num_scaled, y_train)\ncoef = pd.Series(reg.coef_, index = X_train_num_scaled.columns)\ncoef '

In [213]:
""" print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables") """

' print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables") '

In [214]:
""" coef.sort_values() """

' coef.sort_values() '

In [215]:
""" plot_importance(coef,'Lasso') """

" plot_importance(coef,'Lasso') "

# Build a simple model and assess the performance

In [222]:
""" dt = DecisionTreeClassifier().fit(X_train, y_train)
predictions_train = dt.predict(X_train)
predictions_val = dt.predict(X_validation) """

ValueError: could not convert string to float: '30/07/2022'

In [217]:
""" print(dt.score(X_train, y_train))
print(dt.score(X_validation, y_validation)) """

' print(dt.score(X_train, y_train))\nprint(dt.score(X_validation, y_validation)) '

In [218]:
""" print(confusion_matrix(y_train, predictions_train))
print(confusion_matrix(y_validation, predictions_val)) """

' print(confusion_matrix(y_train, predictions_train))\nprint(confusion_matrix(y_validation, predictions_val)) '

In [219]:
""" drugs_2classify['DrugPlant'] = dt.predict(drugs_2classify)
drugs_2classify

#export test data predictions
drugs_2classify['DrugPlant'].to_csv('Exercise1_predictions.csv') """

" drugs_2classify['DrugPlant'] = dt.predict(drugs_2classify)\ndrugs_2classify\n\n#export test data predictions\ndrugs_2classify['DrugPlant'].to_csv('Exercise1_predictions.csv') "